In [4]:
from tensorflow.keras.datasets import mnist
import torch
import torch.nn as nn
import torch.functional as Fun
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [5]:
device = torch.accelerator.current_accelerator().type() if torch.accelerator.is_available() else "cpu"

print(f"Using {device} device")

Using cpu device


In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [10]:
x_train = x_train / 255.0

In [16]:
# Convertir a float32 y normalizar entre 0 y 1
x_train_tensor = torch.tensor(x_train, dtype=torch.float32) / 255.0
x_test_tensor = torch.tensor(x_test, dtype=torch.float32) / 255.0

# Añadir dimensión de canal (N, 1, 28, 28)
x_train_tensor = x_train_tensor.unsqueeze(1)
x_test_tensor = x_test_tensor.unsqueeze(1)

# Convertir etiquetas a long
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [17]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.LogSoftmax() # Utilitzacio de LogSoftMax perque es mes rapida i te millors propietats numeriques
        )

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        logits = self.linear_relu_stack(x)
        return logits

In [18]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
    (5): LogSoftmax(dim=None)
  )
)


In [19]:
NUM_EPOCHS = 32

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:

for epoch in range(NUM_EPOCHS):
    optimizer.zero_grad()  # A. Netejem gradients antics (molt important!)

    outputs = model(x_train_tensor)

    loss = criterion(outputs, y_train_tensor)

    loss.backward()        # B. Calculem els nous gradients (derivades)
    optimizer.step()  

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

c:\Users\xavif\Documents\MASTER\MNIST\.venv\Lib\site-packages\torch\nn\modules\module.py:1776: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch 1, Loss: 2.3027
Epoch 2, Loss: 2.3021
Epoch 3, Loss: 2.3015
Epoch 4, Loss: 2.3010
Epoch 5, Loss: 2.3007
Epoch 6, Loss: 2.3003
Epoch 7, Loss: 2.3000
Epoch 8, Loss: 2.2996
Epoch 9, Loss: 2.2992
Epoch 10, Loss: 2.2988
Epoch 11, Loss: 2.2983
Epoch 12, Loss: 2.2977
Epoch 13, Loss: 2.2971
Epoch 14, Loss: 2.2963
Epoch 15, Loss: 2.2955
Epoch 16, Loss: 2.2945
Epoch 17, Loss: 2.2934
Epoch 18, Loss: 2.2921
Epoch 19, Loss: 2.2907
Epoch 20, Loss: 2.2892
Epoch 21, Loss: 2.2874
Epoch 22, Loss: 2.2855
Epoch 23, Loss: 2.2833
Epoch 24, Loss: 2.2809
Epoch 25, Loss: 2.2783
Epoch 26, Loss: 2.2754
Epoch 27, Loss: 2.2722
Epoch 28, Loss: 2.2687
Epoch 29, Loss: 2.2649
Epoch 30, Loss: 2.2608
Epoch 31, Loss: 2.2563
Epoch 32, Loss: 2.2514


In [ ]:
model.eval()

with torch.no_grad():
    outputs = model(x_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).float().mean()